In [20]:
from pandarallel import pandarallel

pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [53]:
import pandas as pd
df=pd.read_excel('./2022-11-29全部城市.xlsx')
df.to_hdf('./covidtata.h5',key='data1129')

In [55]:
import pandas as pd
import datetime
coviddata=pd.read_hdf('./covidtata.h5',key='data1129')

#get daily covid cases
coviddata['new_cases_daily']=coviddata['新增无症状']+coviddata['新增本土确诊']
df_covid=coviddata[['省份', '城市', '日期','new_cases_daily']]
s1=df_covid['日期']>"2022-02-01" #2022年2月1号开始区分 新增本土无症状（即表里的新增无症状）和新增本土确诊
s2=df_covid['省份'].isin(['北京','上海','天津','重庆'])
s3=~df_covid['城市'].isin(['北京','上海','天津','重庆'])
df_covid=df_covid[s1]
df_covid=df_covid[~(s2&s3)] #去掉直辖市分区的统计，因为后面没抓到其实，而且分析也没有意义

#get weekly covid cases
import datetime
df_covid['日期']=pd.to_datetime(df_covid['日期'])

def getyr(d):
    return str(d.isocalendar()[0])

def getwk(d):
    return str(d.isocalendar()[1]).zfill(2)

df_covid['yr']=df_covid['日期'].apply(getyr)
df_covid['wk']=df_covid['日期'].apply(getwk)

dates=df_covid[['日期','yr','wk']]

df_covid['日期'] = pd.to_datetime(df_covid['日期']).dt.date

#rolling average
df_covid=df_covid.sort_values(by='日期',ascending=True).reset_index(drop=True)
df_covid['last7d_max'] = df_covid.groupby('城市')['new_cases_daily'].transform(lambda x: x.rolling(7, 1).max())
df_covid

# df_covid=df_covid.groupby(by=['省份','城市','yr','wk']).max().reset_index()
# df_covid=df_covid.rename(columns={"new_cases_daily":"new_cases_daily_avg"})

# df_covid=df_covid[['省份','城市','yr','wk','new_cases_daily_avg']]

# dates=dates.drop_duplicates().sort_values(by='日期').reset_index(drop=True)
# dates=dates.drop_duplicates(subset=['yr','wk'],keep='last')


C:\Users\Leo\AppData\Local\Temp\ipykernel_6292\3003474819.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_covid=df_covid[~(s2&s3)] #去掉直辖市分区的统计，因为后面没抓到其实，而且分析也没有意义


,省份,城市,日期,new_cases_daily,yr,wk,last7d_max
0,宁夏,中卫,2022-02-02,0,2022,05,0.0
1,贵州,毕节,2022-02-02,0,2022,05,0.0
2,河南,驻马店,2022-02-02,0,2022,05,0.0
3,广西,防城港,2022-02-02,0,2022,05,0.0
4,河南,南阳,2022-02-02,0,2022,05,0.0
...,...,...,...,...,...,...,...
104031,河南,周口,2022-11-28,0,2022,48,0.0
104032,广西,钦州,2022-11-28,2,2022,48,7.0
104033,广西,南宁,2022-11-28,23,2022,48,47.0
104034,甘肃,金昌,2022-11-28,0,2022,48,6.0


In [419]:
# get weekly amap traffic

amap=pd.read_hdf('./amap.h5',key='raw')
amap=pd.melt(amap,id_vars=['城市','省','城市线'],var_name='date', value_name='traffic')

def getweeknum(d):
    return "Y%s-W%s" % (d.isocalendar()[0],str(d.isocalendar()[1]).zfill(2))

def getyr(d):
    return str(d.isocalendar()[0])

def getwk(d):
    return str(d.isocalendar()[1]).zfill(2)

amap['week']=amap['date'].apply(getweeknum)
amap['yr']=amap['date'].apply(getyr)
amap['wk']=amap['date'].apply(getwk)


amap=amap.groupby(by=['省','城市','yr','wk']).mean().reset_index()
amap=amap.rename(columns={"traffic":"traffic_daily_avg"})


amap_2019=amap[amap['yr'].str.contains('2019')]
amap_2019=amap_2019[['省','城市','wk','traffic_daily_avg']]
amap_2019=amap_2019.groupby(by=['省','城市','wk']).mean().reset_index()
amap_2019=amap_2019.rename(columns={"traffic_daily_avg":"traffic_19_samemonth"})


amap_2022=amap[amap['yr'].str.contains('2022')]

amap_2022=amap_2022.merge(amap_2019,how='left',on=['省','城市','wk'])

amap_2022=amap_2022.rename(columns={"省":"省份"})


In [335]:
# get weekly subway traffic
subway=pd.read_hdf('./subway.h5',key='raw')
subway=pd.melt(subway,id_vars=['指标名称','城市','单位','频率','来源',],var_name='date', value_name='traffic')
subway=subway[['城市','date','traffic']]
subway=subway.dropna()
subway


amap=subway

def getweeknum(d):
    return "Y%s-W%s" % (d.isocalendar()[0],str(d.isocalendar()[1]).zfill(2))

def getyr(d):
    return str(d.isocalendar()[0])

def getwk(d):
    return str(d.isocalendar()[1]).zfill(2)


amap['week']=amap['date'].apply(getweeknum)
amap['yr']=amap['date'].apply(getyr)
amap['wk']=amap['date'].apply(getwk)

amap=amap.groupby(by=['城市','yr','wk']).mean().reset_index()
amap=amap.rename(columns={"traffic":"traffic_daily_avg"})


amap_2019=amap[amap['yr'].str.contains('2019')]
amap_2019=amap_2019[['城市','wk','traffic_daily_avg']]
amap_2019=amap_2019.groupby(by=['城市','wk']).mean().reset_index()
amap_2019=amap_2019.rename(columns={"traffic_daily_avg":"traffic_19_samemonth"})


amap_2022=amap[amap['yr'].str.contains('2022')]

amap_2022=amap_2022.merge(amap_2019,how='left',on=['城市','wk'])
amap_2022
# amap_2022=amap_2022.rename(columns={"省":"省份"})

,城市,yr,wk,traffic_daily_avg,traffic_19_samemonth
0,上海,2022,01,960.157143,927.000000
1,上海,2022,02,1004.085714,1041.671429
2,上海,2022,03,948.200000,1071.200000
3,上海,2022,04,875.528571,1052.400000
4,上海,2022,05,415.185714,878.857143
...,...,...,...,...,...
1325,长沙,2022,43,173.804286,109.387143
1326,长沙,2022,44,172.381429,111.770000
1327,长沙,2022,45,168.410000,111.971429
1328,长沙,2022,46,171.077143,107.470000


In [1]:
# dfo=amap_2022.merge(df_covid,on=['城市','yr','wk'],how='left')

dfo=df_covid
city_popu=pd.read_hdf('./citypopu.h5',key='raw')
city_popu['城市']=city_popu['城市'].str.replace('市',"")
city_popu['常住人口']=pd.to_numeric(city_popu['常住人口'].str.replace(',',''))

dfo=dfo.merge(city_popu,on='城市',how='left')

c1=~dfo['城市'].isin(['伊犁','中国香港'])
c2=dfo['last7d_max']>=0

dfo=dfo[c1 &c2]

def catgorize_per_capita(num):
    num=float(num)
    level=-2
    if num ==0:
        level=-1
    if 0 < num <=1:
        level =0
    if num > 1 and num <=5:
        level =1
    if num > 5 and num <=10:
        level =1
    if num > 10 and num <=50:
        level =1
    if num > 50 and num <=100:
        level =1
    if num > 100:
        level =2
    return level


def catgorize(num):
    num=float(num)
    level=-2
    if num ==0:
        level=-1
    if 0 < num <=1:
        level =0
    if num > 1 and num <=10:
        level =0
    if num > 10 and num <=100:
        level =1
    if num > 100 and num <=500:
        level =2
    if num > 500 and num <=1000:
        level =2
    if num > 1000:
        level =2
    return level

# dfo['cases_daily_per_capita']=dfo['new_cases_daily_avg']/(dfo['常住人口']/100)
# dfo['covid_level_per_capita']=dfo['cases_daily_per_capita'].apply(catgorize_per_capita)


dfo['covid_level']=dfo['last7d_max'].apply(catgorize)
# dfo['traffic_change']=dfo['traffic_daily_avg']/dfo['traffic_19_samemonth']-1

# dfo=dfo.merge(dates,how='left',on='wk')

show=dfo

# c1=show['wk']>"0"
c2=show['covid_level']>-2
# show[c1&~c2].sort_values(by='wk').to_clipboard()

# show[c2].sort_values(by='wk').to_clipboard()

# show.to_clipboard()

import numpy as np
show_value='covid_level'



topivot=show[c2].groupby(by=['日期',show_value]).sum().reset_index()
# topivot2=topivot.merge(dates,how='left',on='wk')
# pd.pivot_table(topivot, values='常住人口', index=['日期'],columns=[show_value], aggfunc=np.sum).to_clipboard()


pd.pivot_table(topivot, values='常住人口', index=['日期'],columns=[show_value], aggfunc='count').to_clipboard()



# topivot=show[c1&c2].groupby(by=['日期',show_value]).mean().reset_index()
# pd.pivot_table(topivot, values='traffic_change', index=['日期'],columns=[show_value], aggfunc=np.sum).to_clipboard()

# topivot=show[c1&c2].groupby(by=['wk',show_value]).count().reset_index()
# topivot2=topivot.merge(dates,how='left',on='wk')
# pd.pivot_table(topivot2, values='traffic_change', index=['日期'],columns=[show_value], aggfunc=np.sum).to_clipboard()



NameError: name 'df_covid' is not defined

In [428]:
show[c1&c2].to_clipboard()